# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

##### <div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [17]:
import pandas as pd
import numpy as np
from scipy import stats
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

##### First examine the data structure for race and call

##### Using pd describe() we can see the basic stats, including mean and std for both races.

In [52]:
data.call[data.race == 'w'].describe()

count    2435.000000
mean        0.096509
std         0.295346
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64

In [53]:
data.call[data.race == 'b'].describe()

count    2435.000000
mean        0.064476
std         0.245649
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64

Note above that we have the same count of black and white rows in the df.  We can also see the mean of white callback is larger than the mean of black callback.

## Question 1 - What test is appropriate for this problem? Does CLT apply?

We can see that for n = 2435 for each race that central limit theorem will apply for our categorical data because the sample size is large enough that the sampling distribution is approaching a normal distribution.  With such a large sample size a z-test for equality of means (b vs w) would be most appropriate.  

## Question 2 - What are the null and alternate hypotheses?

2-sample z-test:

Ho: The callback rate (mean) of black and white applicants will show NO signficant difference

Ha: The callback rate of black and white applicants is will show a signficant difference

alpha = beta = 0.05


In [47]:
t_test = stats.ttest_ind(df.call[df.race == 'b'],df.call[df.race == 'w'])
t_test

Ttest_indResult(statistic=-4.1147052667231092, pvalue=3.9408025140692845e-05)

The pvalue 3.95e-5 is significantly less than our 2-sided 0.025 and thus we reject Ho and conclude that the callback rates are not the same based on racially biased naming.

## 3. Compute margin of error, confidence interval, and p-value.

In [66]:
#create two df's for black and white races
df = pd.DataFrame([data.call,data.race])
df = df.transpose()
df_b = df[df.race == 'b']
df_w = df[df.race == 'w']
df_b.head()

,call,race
2,0,b
3,0,b
7,0,b
8,0,b
9,0,b


In [75]:
#calculate means and stdev of black and white and confirm they match our previous values
mean_b = sum(df_b.call)/len(df_b.call)
mean_w = sum(df_w.call)/len(df_w.call)
mean_b, mean_w

(0.06447638603696099, 0.09650924024640657)

In [79]:
std_b = std(df_b.call)
std_w = std(df_w.call)
std_b, std_w

(0.24559963697158382, 0.29528834517039093)

In [93]:
z_crit = stats.norm.ppf(q = 0.975) #z_crit for two-tail test (0.025 to each side)

margin_of_error = z_crit* math.sqrt((mean_w*(1-mean_w) / len(df_w.call)) + (mean_b*(1-mean_b) / len(df_b.call)))
print('Margin of Error =', margin_of_error)

Margin of Error = 0.0152551260282


In [95]:
print ('lower confidence interval is given by :', (mean_w - mean_b - z_crit * margin_of_error))
print ('upper confidence interval is given by :', (mean_w - mean_b + z_crit * margin_of_error))

lower confidence interval is given by : 0.00213335661452
upper confidence interval is given by : 0.0619323518044


In [110]:
from statsmodels.stats.proportion import proportions_ztest as pzt
ztest, p_val = pzt(np.array([sum(df_b.call), sum(df_w.call)]) , np.array([len(df_b), len(df_w)]), value = 0)
print ('the p-value for the z-test is =', p_val)

the p-value for the z-test is = 3.98388683759e-05


 ### 4. Write a story describing the statistical significance in the context or the original problem.

##### The p-value is <<< than 0.05 and therefore we reject the null hypothesis Ho.  This means that there alternative is considered to be likely in that white sounding names will get more callbacks than black sounding names.

### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

##### We are concluding that race/name is an important factor but we do not yet know if it is the most important factor to callback success because we limited our investigation to only these data values.  if we wanted to amend the analysis we could look at every type of column in the dataset to determine if some other variable had a larger impact on call success.